<a href="https://colab.research.google.com/github/na23150-netizen/LLM-Movie-Recommender/blob/main/LLM_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!pip install langchain langchain-community langchain-huggingface
!pip install chromadb
!pip install sentence-transformers
!pip install gradio
!pip install transformers accelerate


In [ ]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle (1).json


In [ ]:
!kaggle datasets download -d tmdb/tmdb-movie-metadata
!unzip tmdb-movie-metadata.zip


Dataset URL: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata
License(s): other
tmdb-movie-metadata.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  tmdb-movie-metadata.zip
replace tmdb_5000_credits.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace tmdb_5000_movies.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: tmdb_5000_movies.csv    


In [ ]:
import pandas as pd

df = pd.read_csv("tmdb_5000_movies.csv")
df = df[['title', 'genres', 'overview']].dropna()

df['content'] = (
    "Title: " + df['title'] +
    "\nGenres: " + df['genres'] +
    "\nOverview: " + df['overview']
)

df.head()


,title,genres,overview,content
0,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...","Title: Avatar\nGenres: [{""id"": 28, ""name"": ""Ac..."
1,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","Captain Barbossa, long believed to be dead, ha...",Title: Pirates of the Caribbean: At World's En...
2,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",A cryptic message from Bond’s past sends him o...,"Title: Spectre\nGenres: [{""id"": 28, ""name"": ""A..."
3,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",Following the death of District Attorney Harve...,"Title: The Dark Knight Rises\nGenres: [{""id"": ..."
4,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","John Carter is a war-weary, former military ca...","Title: John Carter\nGenres: [{""id"": 28, ""name""..."


In [ ]:
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings


embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


documents = [
    Document(page_content=text, metadata={"title": title})
    for text, title in zip(df["content"], df["title"])
]


db = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
)

print("Vector DB created with:", len(documents), "documents.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Vector DB created with: 4800 documents.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "microsoft/phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)


`torch_dtype` is deprecated! Use `dtype` instead!


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

In [ ]:
def generate_response(query):

    results = db.similarity_search(query, k=4)
    context = "\n\n".join([r.page_content for r in results])


    prompt = f"""
You are a movie recommendation assistant.
User request: {query}

Here are movie plot summaries from a database:
{context}

Give 3–5 movie recommendations with a one sentence explanation each.
Make it friendly and simple.
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=250)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
import gradio as gr

def gradio_chat(query):
    return generate_response(query)

ui = gr.Interface(
    fn=gradio_chat,
    inputs=gr.Textbox(label="What kind of movie do you want?"),
    outputs=gr.Textbox(
        label="Recommendations",
        lines=20,
        max_lines=100,
    ),
    title="🎬 LLM-Powered Movie Recommender",
    description="Built using LangChain, Phi-3 Mini, and ChromaDB."
)

ui.launch()
